In [4]:
import pandas as pd
import numpy as np
import pickle

In [5]:
data_pos = pd.read_csv('positive.csv', delimiter=';', header=None)
data_neg = pd.read_csv('negative.csv', delimiter=';', header=None)

In [6]:
data_pos = data_pos[3]
data_neg = data_neg[3]
data_pos = pd.DataFrame(data_pos)
data_neg = pd.DataFrame(data_neg)
data_pos = data_pos.rename(index=int, columns={3: "text"})
data_neg = data_neg.rename(index=int, columns={3: "text"})
data_pos['labels'] = 1
data_neg['labels'] = 0

In [7]:
len(data_pos), len(data_neg)

(114911, 111923)

In [21]:
import re
import nltk
from nltk import (sent_tokenize as splitter,wordpunct_tokenize as tokenizer)
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def tokenize(text):
    return [tokenizer(sentence) for sentence in splitter(text)]

def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

def tokenize_flatten_df(row, field):
    return flatten(tokenize(row[field]))

def remove_urls(text):
    return re.sub(r"(https?\://)\S+", "", text)

def remove_mentions(text):
    return re.sub(r"@[^:| ]+:? ?", "", text)

def remove_rt(text):
    if text.lower().startswith("rt"):
        return text[3:].strip()
    return text

def remove_urls_mentions_rt_df(row, field):
    return remove_rt(remove_mentions(remove_urls(row[field])))

def replace_hashtags_from_list(tokens_list):
    return [token for token in tokens_list if token != "#"]

def remove_digits(tokens_list):
    return [token for token in tokens_list 
                if not re.match(r"[-+]?\d+(\.[0-9]*)?$", token)]

def remove_containing_non_alphanum(tokens_list):
    return [re.sub(r'[^а-яА-Я\(\)\:]', "", token) for token in tokens_list]
                
def lowercase_list(tokens_list):
    return [token.lower() for token in tokens_list]

def remove_stopwords(tokens_list):
    return [token for token in tokens_list
                if not token in stopwords.words(u'russian')]

def clean_tokens(row, field):
    return replace_hashtags_from_list(
        remove_digits(
            remove_containing_non_alphanum(
                lowercase_list(remove_stopwords(row[field])))))

[nltk_data] Downloading package punkt to /home/stgmadina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/stgmadina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
data = pd.concat([data_pos, data_neg], ignore_index=True)

In [23]:
data['text_cleaned_from_url_mentions_rt'] = \
    data.apply(
        lambda row: remove_urls_mentions_rt_df (row, 'text'),
        axis=1)

data['text_tokenized'] = \
    data.apply(
        lambda row:
            tokenize_flatten_df (row, 'text_cleaned_from_url_mentions_rt'),
        axis=1)

data['text_tokenized_cleaned'] = \
    data.apply(
        lambda row:
            clean_tokens (row, 'text_tokenized'),
        axis=1)

In [24]:
data.head()

,text,labels,text_cleaned_from_url_mentions_rt,text_tokenized,text_tokenized_cleaned
0,"@first_timee хоть я и школота, но поверь, у на...",1,"хоть я и школота, но поверь, у нас то же самое...","[хоть, я, и, школота, ,, но, поверь, ,, у, нас...","[школота, , поверь, , самое, :, , общество, пр..."
1,"Да, все-таки он немного похож на него. Но мой ...",1,"Да, все-таки он немного похож на него. Но мой ...","[Да, ,, все, -, таки, он, немного, похож, на, ...","[да, , , таки, немного, похож, , но, мальчик, ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,Ну ты идиотка) я испугалась за тебя!!!,"[Ну, ты, идиотка, ), я, испугалась, за, тебя, ...","[ну, идиотка, ), испугалась, , ]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"""Кто то в углу сидит и погибает от голода, а м...","["", Кто, то, в, углу, сидит, и, погибает, от, ...","[, кто, углу, сидит, погибает, голода, , ещ, ,..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"Вот что значит страшилка :D\nНо блин,посмотрев...","[Вот, что, значит, страшилка, :, D, Но, блин, ...","[вот, значит, страшилка, :, , но, блин, , посм..."


In [25]:
data = data.drop(['text', 'text_tokenized', 'text_cleaned_from_url_mentions_rt'], axis=1)
data.columns = ['label', 'text']

In [26]:
data = data[data['text'].map(lambda d: len(d)) > 0]

In [27]:
data['text'] = [[i for i in d if i!=''] for d in data['text']]
data['text'] = [" ".join(d) for d in data['text']]
data['text'] = [d.strip() for d in data['text']]

In [39]:
empty_rows = []
for i , t in enumerate(data['text'].values):
    if t == '':
        empty_rows.append(i)

In [40]:
data.drop(data.index[empty_rows], inplace=True)

In [30]:
data['text_tokenized'] = \
    data.apply(
        lambda row:
            tokenize_flatten_df (row, 'text'),axis=1)

In [31]:
data = data.drop(['text'], axis=1)
data.columns = ['label', 'text']

In [33]:
with open('twitter_prep_data_brackets.pickle', 'wb') as f:
        pickle.dump(data, f)